In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers
#import efficientnet.tfkeras as efn
k=16

In [2]:
#读取数据集
total_data=pd.read_pickle("1Mdataset.pkl")

In [3]:
#历史行为图像
hist_array=np.load('hist_array.npy')

In [4]:
#只取其中3张图像
hist_array.shape

(723783, 3, 128)

In [5]:
#目标物品图像
target_array=np.load('target_img.npy')

In [6]:
total_data

,UserID,MovieID,Rating,timestamps,Gender,Age,OccupationID,Zip-code,Title,Genres,style1,style2,style3,time,history
0,1722,944,1,20.697645,1,3,0,2274,"Shining, The (1980)",Horror,10,12,10,59,"[1857, 415, 554, 1228, 2732, 2737, 2609, 1746,..."
1,5999,1018,1,20.679154,0,2,1,1938,Cape Fear (1962),Film-Noir|Thriller,9,15,10,41,"[471, 934, 871, 2472, 714, 1515, 2189, 946, 16..."
2,4308,1858,0,20.687909,1,2,0,1226,Blast from the Past (1999),Comedy|Romance,4,13,10,78,"[98, 517, 1421, 2798, 783, 369, 2772, 949, 277..."
3,33,2323,1,20.702025,1,4,3,1915,Awakenings (1990),Drama,7,12,10,69,"[1280, 33, 2595, 1965, 1906, 1554, 1332, 1013,..."
4,3732,2215,1,20.766544,1,2,17,2030,Crimes and Misdemeanors (1989),Comedy,4,12,10,68,"[901, 198, 2215, 680, 265, 303, 725, 2838, 726..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723778,1835,1758,1,20.697823,1,2,19,493,"Money Pit, The (1986)",Comedy,4,12,10,65,"[1408, 547, 2213, 2344, 782, 817, 983, 1049, 5..."
723779,5572,2685,1,20.681717,0,2,4,2146,"Man with the Golden Gun, The (1974)",Action,0,12,10,53,"[2242, 517, 264, 682, 458, 686, 913, 274, 885,..."
723780,5364,469,1,20.682989,0,2,3,1903,Sleepless in Seattle (1993),Comedy|Romance,4,13,10,72,"[1729, 2435, 1156, 1771, 1485, 1778, 660, 696,..."
723781,4055,113,0,20.688149,1,5,0,3416,Down Periscope (1996),Comedy,4,12,10,75,"[76, 15, 497, 1715, 52, 822, 342, 505, 1243, 669]"


In [7]:
print(hist_array.shape)
print(target_array.shape)

(723783, 3, 128)
(723783, 1, 128)


In [8]:
#u_input = Input([4,])
#u_input[:,3]

In [9]:
#u_input.shape[1]

In [10]:
#for i in range(u_input.shape[1]):
#    print(i)

In [11]:
#用户侧输入处理
user_feats=['Gender','Age','OccupationID','Zip-code']
u_input = Input([4,])
user_kd_embed = []
for i in range(u_input.shape[1]):
    f = user_feats[i]
    print(f)
    voc_size = total_data[f].nunique()
    print(voc_size)
    _embed = Embedding(voc_size+1, k,embeddings_regularizer=regularizers.l2(0.0001))(u_input[:,i])
    _embed=tf.expand_dims(_embed,1)
    user_kd_embed.append(_embed)
    print(_embed)
tu=user_kd_embed

Gender
2
Tensor("ExpandDims:0", shape=(None, 1, 16), dtype=float32)
Age
7
Tensor("ExpandDims_1:0", shape=(None, 1, 16), dtype=float32)
OccupationID
21
Tensor("ExpandDims_2:0", shape=(None, 1, 16), dtype=float32)
Zip-code
3439
Tensor("ExpandDims_3:0", shape=(None, 1, 16), dtype=float32)


In [12]:
tu

[<tf.Tensor 'ExpandDims:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'ExpandDims_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'ExpandDims_2:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'ExpandDims_3:0' shape=(None, 1, 16) dtype=float32>]

In [13]:
total_data['style1'].nunique()

18

In [14]:
#项目侧输入处理
item_feats=['time','style1','style2','style3']
i_input = Input([4,])
item_kd_embed = []
for i in range(i_input.shape[1]):
    f = item_feats[i]
    print(f)
    voc_size = total_data[f].nunique()
    print(voc_size)
    _embed = Embedding(voc_size+1, k,embeddings_regularizer=regularizers.l2(0.0001))(i_input[:,i])
    _embed=tf.expand_dims(_embed,1)
    item_kd_embed.append(_embed)
temp=Add()([item_kd_embed[1],item_kd_embed[2],item_kd_embed[3]])
temp=temp/3
i_embed=[item_kd_embed[0],temp]
ti=i_embed
#target_id=item_inputs[0]

time
190
style1
18
style2
18
style3
16


In [15]:
ti

[<tf.Tensor 'ExpandDims_4:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'truediv:0' shape=(None, 1, 16) dtype=float32>]

In [16]:
#tt=tu+ti

In [17]:
tu=Concatenate(axis=1)(tu)
ti=Concatenate(axis=1)(ti)

In [18]:
tu

<tf.Tensor 'concatenate/Identity:0' shape=(None, 4, 16) dtype=float32>

In [19]:
ti

<tf.Tensor 'concatenate_1/Identity:0' shape=(None, 2, 16) dtype=float32>

In [20]:
#用户历史图像序列
hist_inputs = Input([3,128], name='hist_img')

In [21]:
#目标图片
target_img = Input([1,128], name='target_img')

In [22]:
target_img

<tf.Tensor 'target_img:0' shape=(None, 1, 128) dtype=float32>

# 视觉偏好提取网络

In [23]:
#生成Query、Key、Value向量
class qkv(tf.keras.layers.Layer):
    def __init__(self,d):
        super(qkv,self).__init__() 
        self.d=d
    def build(self,input_shape):
        self.kernel=self.add_variable(shape=[input_shape[-1],self.d])
    def call(self,input):
        embed = tf.matmul(input, self.kernel)
        return embed

In [24]:
#QKV注意力机制，目标物品图像的Q乘以用户历史行为序列中图像的K，算出注意力分数，并按照注意力分数对V进行池化
def Image_TF(hist_embed,target_embed,d):
    Q_net=qkv(d)
    K_net=qkv(d)
    V_net=qkv(d)
    embed_q=Q_net(target_embed)
    embed_k=K_net(hist_embed)
    embed_v=V_net(hist_embed)
    energy = tf.matmul(embed_q, tf.transpose(embed_k, [0, 2, 1]))
    attention = Softmax(axis=-1)(energy)
    attention_output = tf.matmul(attention, embed_v)
    output=LeakyReLU()(attention_output)
    return output

In [25]:
#用户视觉偏好向量
visual_performance=Image_TF(hist_inputs,target_img,16)
visual_performance

Instructions for updating:
Please use `layer.add_weight` method instead.


<tf.Tensor 'leaky_re_lu/Identity:0' shape=(None, 1, 16) dtype=float32>

# 用户侧特征交叉

In [26]:
user_map=Concatenate(axis=1)([tu,visual_performance])

In [27]:
user_map

<tf.Tensor 'concatenate_2/Identity:0' shape=(None, 5, 16) dtype=float32>

In [28]:
#特征增强网络，将其投影到更高维度，增强特征向量表达能力
class embed_net(tf.keras.layers.Layer):
    def __init__(self,d):
        super(embed_net,self).__init__() 
        self.d=d
    def build(self,input_shape):
        self.kernel=self.add_variable(shape=[input_shape[-1],self.d])
    def call(self,input):
        embed = tf.matmul(input, self.kernel)
        return embed

In [29]:
#注意力网络
class AFM(tf.keras.layers.Layer):
    def __init__(self):
        super(AFM,self).__init__() 
    def build(self,input_shape):
        self.kernel=self.add_variable(shape=[input_shape[1],input_shape[2],input_shape[-1]])
    def call(self,input):
        embed = tf.multiply(input, self.kernel)
        return embed

In [30]:
#特征融合与交互层
def MAFM(embed_map,d,n_attention_head,temp_x0):
    k =embed_map.shape[1]
    embedl=embed_net(d)
    embed_xl=embedl(embed_map)
    attention_heads=[]
    for i in range(n_attention_head):
        AFM_net=AFM()
        output= AFM_net(temp_x0)  
        output=tf.reduce_sum(output,2)
        output=Reshape(target_shape=(k,d))(output) 
        output=tf.multiply(output,embed_xl)
        attention_heads.append(output)
    output=Concatenate(axis=-1)(attention_heads)  
    output=Dense(d)(output)
    output=LeakyReLU()(output)
    return output

In [31]:
#创建多层特征融合与交互层，并将记录每层输出
def build_MAFM(x0, n_layers,d,n_head):
    k =x0.shape[1]
    temp_x0=[]
    x0_net=embed_net(d)
    new_x0=x0_net(x0)
    #x0=new_x0
    for i in range(k): 
        temp=Reshape(target_shape=(1,k,d))(new_x0)
        temp_x0.append(temp)
    temp_x0=Concatenate(axis=1)(temp_x0)
    xl = x0
    out=[]
    for i in range(n_layers):
        xl = MAFM(xl,d,n_head,temp_x0)
        out.append(xl)
    out=Concatenate(axis=1)(out)
    return out

In [32]:
uu=build_MAFM(user_map,3,24,2)

In [33]:
#用户侧特征高阶注意力融合表示
uu

<tf.Tensor 'concatenate_7/Identity:0' shape=(None, 15, 24) dtype=float32>

# 物品侧特征交叉

In [34]:
target_map=Dense(16,activation='relu')(target_img)

In [35]:
item_map=Concatenate(axis=1)([ti,target_map])

In [36]:
item_map

<tf.Tensor 'concatenate_8/Identity:0' shape=(None, 3, 16) dtype=float32>

In [37]:
ii=build_MAFM(item_map,3,24,2)

In [38]:
##项目侧特征高阶注意力融合表示
ii

<tf.Tensor 'concatenate_13/Identity:0' shape=(None, 9, 24) dtype=float32>

# 输出层

In [39]:
add=Concatenate(axis=1)([uu,ii])
add=Flatten()(add)
#add=Dense(512,activation='relu')(add)
add=Dense(384)(add)
add=tf.keras.layers.LeakyReLU()(add)
#add=Dense(128,activation='relu')(add)
#add=Dense(192,activation='relu')(add)
#add=Dense(64,activation='relu')(add)
#add=BatchNormalization()(add)
#add=Dense(512,activation='relu')(add)
#add=Dense(256,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(add)
#add=Dropout(256,activation='relu',)
#add=Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(add)
#add=Dense(64,activation='relu')(add)
output_layer = Dense(1, activation='sigmoid')(add)
#完成数据流，输入、输出
model=Model([u_input,i_input,hist_inputs,target_img], output_layer)

In [40]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
hist_img (InputLayer)           [(None, 3, 128)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
target_img (InputLayer)         [(None, 1, 128)]     0                                            
__________________________________________________________________________________________________
qkv_1 (qkv)                     (None, 3, 16)        2048        hist_img[0][0]                   
______________________________________________________________________________________________

In [41]:
#处理用户侧数据集
u_input=np.asarray([total_data[f].values for f in user_feats])
u_input=np.transpose(u_input,(1,0))
u_input.shape

(723783, 4)

In [42]:
#处理物品侧数据集
i_input=np.asarray([total_data[f].values for f in item_feats])
i_input=np.transpose(i_input,(1,0))
i_input.shape

(723783, 4)

In [43]:
#用户历史图像与目标物品图像
h_input=hist_array
t_input=target_array

In [44]:
t_input.shape

(723783, 1, 128)

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
#分割训练集与测试集
#split = train_test_split(u_input,i_input,h_input,t_input,total_data['Rating'].values,test_size=0.25, random_state=42)
split = train_test_split(u_input,i_input,h_input,t_input,total_data['Rating'].values,test_size=0.25)

In [47]:
(Tu_input,Vu_input,Ti_input,Vi_input,Th_input,Vh_input,Tt_input,Vt_input,train_label,val_label) = split

In [48]:
Tu_input.shape
Vu_input.shape
Ti_input.shape
Vi_input.shape
Th_input.shape
Vh_input.shape
Tt_input.shape
Vt_input.shape

(180946, 1, 128)

In [49]:
tbCallBack = TensorBoard(log_dir= 'logs',  # log 目录
                 histogram_freq=0, 
                 write_graph=True,  
                 write_grads=True,
                 write_images=True,
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

In [50]:
#设置损失和评价指标
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, 
              loss="binary_crossentropy", 
              metrics=["binary_crossentropy", tf.keras.metrics.AUC(name='auc')])

In [51]:
#训练
model.fit([Tu_input,Ti_input,Th_input,Tt_input], 
          train_label, epochs=30, batch_size=8192,
          validation_data=([Vu_input,Vi_input,Vh_input,Vt_input], val_label), 
          callbacks=[tbCallBack])

Train on 542837 samples, validate on 180946 samples
Epoch 1/30


D:\anaconda\envs\py37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\anaconda\envs\py37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


542837/542837 [==============================] - 11s 20us/sample - loss: 0.4746 - binary_crossentropy: 0.4662 - auc: 0.7192 - val_loss: 0.4154 - val_binary_crossentropy: 0.4001 - val_auc: 0.8216
Epoch 2/30
542837/542837 [==============================] - 7s 12us/sample - loss: 0.3945 - binary_crossentropy: 0.3805 - auc: 0.8380 - val_loss: 0.3909 - val_binary_crossentropy: 0.3777 - val_auc: 0.8435
Epoch 3/30
542837/542837 [==============================] - 6s 12us/sample - loss: 0.3805 - binary_crossentropy: 0.3671 - auc: 0.8505 - val_loss: 0.3812 - val_binary_crossentropy: 0.3682 - val_auc: 0.8516
Epoch 4/30
542837/542837 [==============================] - 6s 12us/sample - loss: 0.3722 - binary_crossentropy: 0.3584 - auc: 0.8585 - val_loss: 0.3775 - val_binary_crossentropy: 0.3630 - val_auc: 0.8567
Epoch 5/30
542837/542837 [==============================] - 7s 12us/sample - loss: 0.3681 - binary_crossentropy: 0.3531 - auc: 0.8632 - val_loss: 0.3749 - val_binary_crossentropy: 0.3602 - v

In [52]:
#降低学习率再来
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              loss="binary_crossentropy", 
              metrics=["binary_crossentropy", tf.keras.metrics.AUC(name='auc')])

In [ ]:
model.fit([Tu_input,Ti_input,Th_input,Tt_input], 
          train_label, epochs=10, batch_size=8192,
          validation_data=([Vu_input,Vi_input,Vh_input,Vt_input], val_label), 
          callbacks=[tbCallBack])

Train on 542837 samples, validate on 180946 samples
Epoch 1/10


D:\anaconda\envs\py37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\anaconda\envs\py37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


542837/542837 [==============================] - 13s 24us/sample - loss: 0.3133 - binary_crossentropy: 0.2977 - auc: 0.9077 - val_loss: 0.3402 - val_binary_crossentropy: 0.3256 - val_auc: 0.8889
Epoch 2/10
542837/542837 [==============================] - 7s 13us/sample - loss: 0.3030 - binary_crossentropy: 0.2890 - auc: 0.9136 - val_loss: 0.3377 - val_binary_crossentropy: 0.3243 - val_auc: 0.8905
Epoch 3/10
542837/542837 [==============================] - 7s 13us/sample - loss: 0.2987 - binary_crossentropy: 0.2856 - auc: 0.9159 - val_loss: 0.3367 - val_binary_crossentropy: 0.3240 - val_auc: 0.8908
Epoch 4/10
172032/542837 [========>.....................] - ETA: 4s - loss: 0.2939 - binary_crossentropy: 0.2813 - auc: 0.9191

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, 
              loss="binary_crossentropy", 
              metrics=["binary_crossentropy", tf.keras.metrics.AUC(name='auc')])

In [ ]:
model.fit([Tu_input,Ti_input,Th_input,Tt_input], 
          train_label, epochs=30, batch_size=8192,
          validation_data=([Vu_input,Vi_input,Vh_input,Vt_input], val_label), 
          callbacks=[tbCallBack])

In [ ]:
Vu_input=Vu_input.astype(np.float32)
Vi_input=Vi_input.astype(np.float32)

In [ ]:
#模型预测
prediction_result = model.predict([Vu_input,Vi_input,Vh_input,Vt_input])

In [ ]:
prediction_result

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
plot_model(model, "BaseModel.png")